In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import gkey

#### Store Part I results into DataFrame
Load the csv exported in Part I to a DataFrame

In [2]:
# Load csv
city_weather = pd.read_csv('City_Weather.csv')
# Visualize
city_weather.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,rikitea,-23.1203,-134.9692,70.88,67,25,5.03,PF,1627246667
1,puerto ayora,36.5939,-6.2330,75.31,80,0,17.27,ES,1627246763
2,ushuaia,-54.8000,-68.3000,37.06,75,40,20.71,AR,1627246679
3,jamestown,42.0970,-79.2353,84.09,54,20,9.22,US,1627246555
4,georgetown,5.4112,100.3354,82.89,86,20,1.99,MY,1627246413


#### Humidity Heatmap
Configure gmaps.Use the Lat and Lng as locations and Humidity as the weight. Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=gkey)

# Store 'Lat' and 'Lng' into  locations 
locations = city_weather[["Lat", "Lng"]].astype(float)
# Store 'Humidity' into  weight 
humidity = city_weather['Humidity']
# Drop NaN values
city_weather = city_weather.dropna()

# Create Heatmap layer
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=200,
                                 point_radius = 2)
fig.add_layer(heat_layer)
# Visualize
fig

Figure(layout=FigureLayout(height='420px'))

#### Create new DataFrame fitting weather criteria
Narrow down the cities to fit weather conditions.

In [4]:
# Find ideal weather condition with a max temp >=75 and <=80, wind speed <10mph, cloudiness=0
ideal_weather = city_weather[(city_weather['Max Temp'] >= 75) & 
                             (city_weather['Max Temp'] <= 80) &
                             (city_weather['Wind Speed'] < 10) & 
                             (city_weather['Cloudiness'] == 0)]
# Visualize cities with ideal weather
ideal_weather

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
103,marand,38.4329,45.7749,79.14,64,0,7.54,IR,1627246786
120,arraial do cabo,-22.8362,-46.6460,77.52,34,0,3.83,BR,1627246791
305,wajima,37.4000,136.9000,78.96,81,0,2.98,JP,1627246854
438,erzin,36.9559,36.2000,79.00,76,0,1.88,TR,1627246894


#### Hotel Map
Store into variable named hotel_df.Add a "Hotel Name" column to the DataFrame.Set parameters to search for hotels with 5000 meters.Hit the Google Places API for each city's coordinates.Store the first Hotel result into the DataFrame.Plot markers on top of the heatmap.

In [5]:
# Add Hotel Name column to dataframe copy
hotel_df = ideal_weather.copy()
hotel_df['Hotel Name'] = " "

In [6]:
# Use Google Places API to find the first hotel for each city located within 5000 meters of coordinates
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    try:
        # Define parameters
        coordinates = f"{row['Lat']},{row['Lng']}"
        params = {"location": coordinates, "radius": 5000, "type": "lodging", "key": gkey}
        # Call API
        response = requests.get(base_url, params=params)
        # If response is successful, convert response to json
        if response.status_code == 200:
            response = response.json()
            # Add lodging results from json to 'Hotel Name' column in hotel_df
            hotel_df.loc[index, 'Hotel Name'] = response['results'][0]['name']
        #
        else: 
            print("City not found. Skipping...") 
    except Exception as e:
        print("Something went wrong")
        
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
103,marand,38.4329,45.7749,79.14,64,0,7.54,IR,1627246786,کوچه صفا
120,arraial do cabo,-22.8362,-46.6460,77.52,34,0,3.83,BR,1627246791,Pousada da Maricota
305,wajima,37.4000,136.9000,78.96,81,0,2.98,JP,1627246854,Wajiima Onsen Yashio
438,erzin,36.9559,36.2000,79.00,76,0,1.88,TR,1627246894,Artemis Otel


In [7]:
# Plot markers on top of heat map
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

# Add markers
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))